In [4]:
import layoutparser as lp
import cv2

image = cv2.imread("3.png")
image = image[..., ::-1]

In [5]:
model = lp.Detectron2LayoutModel('lp://PrimaLayout/mask_rcnn_R_50_FPN_3x/config',
                                 extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.8],
                                 label_map={1:"TextRegion", 2:"ImageRegion", 3:"TableRegion", 4:"MathsRegion", 5:"SeparatorRegion", 6:"OtherRegion"})

The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std
  proposal_generator.anchor_generator.cell_anchors.{0, 1, 2, 3, 4}


In [6]:
layout = model.detect(image)

/home/pika/.virtualenvs/tifr-ass/lib/python3.8/site-packages/detectron2/structures/image_list.py:99: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/home/pika/.virtualenvs/tifr-ass/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [7]:
print(type(layout))
print(layout[0])

<class 'layoutparser.elements.layout.Layout'>
TextBlock(block=Rectangle(x_1=59.027339935302734, y_1=2205.47412109375, x_2=1041.1748046875, y_2=2246.536865234375), text=None, id=None, type=TextRegion, parent=None, next=None, score=0.9999967813491821)


In [8]:
text_blocks = lp.Layout([b for b in layout if b.type=='TextRegion']) #and b.score>0.91
figure_blocks = lp.Layout([b for b in layout if b.type=='ImageRegion'])

text_blocks = lp.Layout([b for b in text_blocks if not any(b.is_in(b_fig) for b_fig in figure_blocks)])

In [9]:
h, w = image.shape[:2]

left_interval = lp.Interval(0, w/2*1.05, axis='x').put_on_canvas(image)

left_blocks = text_blocks.filter_by(left_interval, center=True)
left_blocks.sort(key = lambda b:b.coordinates[1])

right_blocks = [b for b in text_blocks if b not in left_blocks]
right_blocks.sort(key = lambda b:b.coordinates[1])

text_blocks = lp.Layout([b.set(id = idx) for idx, b in enumerate(left_blocks + right_blocks)])

In [11]:
new_image = lp.draw_box(image, text_blocks,
            box_width=3,
            show_element_id=True)
print(type(new_image))

<class 'PIL.Image.Image'>


In [31]:
i=63
x1, y1, x2, y2 = text_blocks[i].block.x_1, text_blocks[i].block.y_1, text_blocks[i].block.x_2, text_blocks[i].block.y_2
cropped_img = image[int(y1):int(y2), int(x1):int(x2)]
cv2.imwrite("cropped.png", cropped_img)

True